In [1]:
import os
import pymysql

# 測試連線
conn = pymysql.connect(
    host=os.getenv('DB_HOST', 'db'),
    user=os.getenv('DB_USER', 'admin'),
    password=os.getenv('DB_PASSWORD', 'password'),
    database='project_db'
)
print("連線成功！")

連線成功！


In [2]:
# --- M1 初始化單元格 ---
import pandas as pd
from sqlalchemy import create_engine, text
import sqlalchemy
import os

# 1. 建立引擎
engine = create_engine("mysql+pymysql://admin:password@db:3306/project_db")

# 2. 檢查套件是否成功永久安裝
print(f"✅ 套件驗證成功！SQLAlchemy 版本: {sqlalchemy.__version__}")

# 3. 如果資料表是空的，自動匯入 (下次進來只要跑這格就好)
csv_path = "cleaned_data_revise_2.csv"
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.to_sql('temp_raw_data', con=engine, if_exists='replace', index=False)
    print(f"✅ 資料已自動更新，目前總筆數: {len(df)}")

✅ 套件驗證成功！SQLAlchemy 版本: 2.0.45
✅ 資料已自動更新，目前總筆數: 17880


In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import os

# 1. 重新定義 engine (確保變數存在)
DB_URL = "mysql+pymysql://admin:password@db:3306/project_db"
engine = create_engine(DB_URL)

# 2. 設定 CSV 路徑 (既然你放進 backend 了，直接寫檔名即可)
csv_path = "cleaned_data_revise_2.csv"

try:
    # 3. 讀取與匯入
    df_to_import = pd.read_csv(csv_path)
    
    with engine.connect() as conn:
        # 將資料寫入資料庫
        df_to_import.to_sql('temp_raw_data', con=engine, if_exists='replace', index=False)
        print(f"✅ 成功！已匯入 {len(df_to_import)} 筆資料到 temp_raw_data 表中。")
        
        # 4. 立即測試查詢
        df_check = pd.read_sql(text("SELECT * FROM temp_raw_data LIMIT 5"), conn)
        print("📊 資料庫前五筆資料如下：")
        display(df_check)
        
except FileNotFoundError:
    print(f"❌ 找不到檔案：{csv_path}。請確認檔案是否已移動到 backend 資料夾下。")
except Exception as e:
    print(f"❌ 匯入失敗：{e}")

✅ 成功！已匯入 17880 筆資料到 temp_raw_data 表中。
📊 資料庫前五筆資料如下：


,title,telecommuting,has_company_logo,has_questions,employment_type,fraudulent,in_balanced_dataset,country,industry_group,edu_level
0,Marketing Intern,0,1,0,Unknown,0,0,US,Unknown,Unknown
1,Customer Service - Cloud Video Production,0,1,0,Full-time,0,0,NZ,Corporate_Services,Unknown
2,Commissioning Machinery Assistant (CMA),0,1,0,Unknown,0,0,US,Unknown,Unknown
3,Account Executive - Washington DC,0,1,0,Full-time,0,0,US,Tech,Bachelor
4,Bill Review Manager,0,1,1,Full-time,0,0,US,Health_Science,Bachelor


In [3]:
import pandas as pd
from sqlalchemy import text

# 1. 讀取你看到的那個 CSV 檔案
# 注意：在 Container 內部的路徑通常與電腦不同，
# 根據你的 YAML 掛載，試試看這個路徑：
csv_path = "/app/cleaned_data_revise_2.csv"

try:
    df_to_import = pd.read_csv(csv_path)
    
    # 2. 將資料寫入資料庫
    # if_exists='append' 表示把資料加進去，'replace' 表示刪掉舊的換新的
    df_to_import.to_sql('temp_raw_data', con=engine, if_exists='replace', index=False)
    
    print(f"✅ 成功！已匯入 {len(df_to_import)} 筆資料到 temp_raw_data 表中。")
    
    # 3. 再次測試你的聚合查詢
    df_check = pd.read_sql(text("SELECT * FROM temp_raw_data LIMIT 5"), engine)
    display(df_check)
    
except FileNotFoundError:
    print(f"❌ 找不到檔案：{csv_path}，請檢查 Container 內部的實際掛載路徑。")
except Exception as e:
    print(f"❌ 匯入失敗：{e}")

✅ 成功！已匯入 17880 筆資料到 temp_raw_data 表中。


,title,telecommuting,has_company_logo,has_questions,employment_type,fraudulent,in_balanced_dataset,country,industry_group,edu_level
0,Marketing Intern,0,1,0,Unknown,0,0,US,Unknown,Unknown
1,Customer Service - Cloud Video Production,0,1,0,Full-time,0,0,NZ,Corporate_Services,Unknown
2,Commissioning Machinery Assistant (CMA),0,1,0,Unknown,0,0,US,Unknown,Unknown
3,Account Executive - Washington DC,0,1,0,Full-time,0,0,US,Tech,Bachelor
4,Bill Review Manager,0,1,1,Full-time,0,0,US,Health_Science,Bachelor


In [4]:
#顯示目前有的資料表
from sqlalchemy import text

with engine.connect() as connection:
    # 使用 text() 包裝 SQL 字串
    result = connection.execute(text("SHOW TABLES;"))
    # 2.0 版本的結果需要用 mappings() 或直接迭代
    tables = [row[0] for row in result]
    print("目前有的資料表：", tables)

目前有的資料表： ['temp_raw_data']


In [5]:
from sqlalchemy import create_engine, text
import pandas as pd
import os

# 1. 建立引擎
db_user = os.getenv('DB_USER', 'admin')
db_password = os.getenv('DB_PASSWORD', 'password')
db_name = os.getenv('DB_NAME', 'project_db')
DB_URL = f"mysql+pymysql://{db_user}:{db_password}@db:3306/{db_name}"
engine = create_engine(DB_URL)

# 2. 實作聚合查詢 (根據你的欄位修改：統計各產業的職缺數與遠端比例)
def get_industry_stats():
    # 改用 industry_group 作為分類，telecommuting 作為聚合指標
    sql_agg = text("""
    SELECT 
        industry_group, 
        COUNT(*) as job_count, 
        SUM(telecommuting) as remote_count,
        AVG(telecommuting) * 100 as remote_percentage
    FROM temp_raw_data 
    GROUP BY industry_group
    ORDER BY job_count DESC
    """)
    with engine.connect() as connection:
        return pd.read_sql(sql_agg, connection)

# 3. 實作分頁 (Pagination)
def get_paginated_jobs(page=1, size=10):
    offset = (page - 1) * size
    sql_page = text("SELECT * FROM temp_raw_data LIMIT :size OFFSET :offset")
    with engine.connect() as connection:
        return pd.read_sql(sql_page, connection, params={"size": size, "offset": offset})

# --- 執行測試 ---
try:
    print("正在執行【產業別】聚合查詢...")
    df_summary = get_industry_stats()
    print("✅ 查詢成功！")
    display(df_summary.head(10)) # 顯示前 10 筆

    print("\n正在執行分頁查詢 (第 1 頁)...")
    df_page = get_paginated_jobs(page=1, size=5)
    display(df_page)

except Exception as e:
    print(f"❌ 執行失敗: {e}")

正在執行【產業別】聚合查詢...
✅ 查詢成功！


,industry_group,job_count,remote_count,remote_percentage
0,Unknown,5995,243.0,4.0534
1,Tech,4375,317.0,7.2457
2,Corporate_Services,1554,51.0,3.2819
3,Education,1062,10.0,0.9416
4,Finance,1030,47.0,4.5631
5,Consumer_Retail,1000,24.0,2.4000
6,Health_Science,863,20.0,2.3175
7,Industrial_Energy,784,17.0,2.1684
8,Media_Entertainment,585,21.0,3.5897
9,Logistics_Travel,449,9.0,2.0045



正在執行分頁查詢 (第 1 頁)...


,title,telecommuting,has_company_logo,has_questions,employment_type,fraudulent,in_balanced_dataset,country,industry_group,edu_level
0,Marketing Intern,0,1,0,Unknown,0,0,US,Unknown,Unknown
1,Customer Service - Cloud Video Production,0,1,0,Full-time,0,0,NZ,Corporate_Services,Unknown
2,Commissioning Machinery Assistant (CMA),0,1,0,Unknown,0,0,US,Unknown,Unknown
3,Account Executive - Washington DC,0,1,0,Full-time,0,0,US,Tech,Bachelor
4,Bill Review Manager,0,1,1,Full-time,0,0,US,Health_Science,Bachelor


In [9]:
with engine.connect() as connection:
    # 取得欄位詳細資訊
    result = connection.execute(text("DESCRIBE temp_raw_data;"))
    columns = [row[0] for row in result]
    print("📢 你的表真正的欄位名稱是：", columns)

📢 你的表真正的欄位名稱是： ['title', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'fraudulent', 'in_balanced_dataset', 'country', 'industry_group', 'edu_level']


In [7]:
#取得裡面的資料筆數
with engine.connect() as connection:
    result = connection.execute(text("SELECT COUNT(*) FROM temp_raw_data;"))
    count = result.scalar()
    print(f"📊 目前 temp_raw_data 資料表中的總筆數為: {count}")

📊 目前 temp_raw_data 資料表中的總筆數為: 17880


In [ ]:
%pip install sqlalchemy cryptography
#pandas與sqlalchemy版本相容性問題

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.2/590.2 kB 9.3 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
# 下次進來的第一件事：執行此格檢查現況
from sqlalchemy import create_engine, text
import pandas as pd

# 1. 建立連線
engine = create_engine("mysql+pymysql://admin:password@db:3306/project_db")

try:
    with engine.connect() as connection:
        # 2. 檢查資料筆數
        count_res = connection.execute(text("SELECT COUNT(*) FROM temp_raw_data;"))
        count = count_res.scalar()
        
        # 3. 檢查欄位
        col_res = connection.execute(text("DESCRIBE temp_raw_data;"))
        cols = [row[0] for row in col_res]
        
        print(f"✅ 連線狀態：正常")
        print(f"📊 目前資料筆數：{count} (如果是 0，代表 DB 組還沒匯入資料)")
        print(f"📢 目前欄位名稱：{cols}")
        
except Exception as e:
    print(f"❌ 連線失敗或環境重置：{e}")

✅ 連線狀態：正常
📊 目前資料筆數：17880 (如果是 0，代表 DB 組還沒匯入資料)
📢 目前欄位名稱：['title', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'fraudulent', 'in_balanced_dataset', 'country', 'industry_group', 'edu_level']
